<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Tinoco
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/sebatinoco/MDS7202/tree/main/lab7`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# UMAP
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/MDS/Labs/Lab 7/'
except: 
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Leemos los datos, especificando las variables categóricas en base al informe **`Player_Stats_Report.html`** provisto

In [3]:
df_players = pd.read_csv(path + 'stats_players.csv', dtype = {'Name': 'category', 'Nationality': 'category', 'National_Position': 'category', 'Club_Position': 'category', 
                                                              'Preffered_Foot': 'category', 'Work_Rate': 'category', 'Weak_foot': 'category', 'Skill_Moves': 'category'})
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [4]:
pd.DataFrame(np.where(pd.isnull(df_players['National_Position']), 0, 1)).value_counts()

0    16513
1     1075
dtype: int64

A partir del conteo de clases, se observa que existe un desbalanceo en las clases. En particular, se observa que los seleccionados nacionales ascienden a 1075 filas del dataset, mientras que los no seleccionados son 16.513.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [5]:
target = 'National_Position' # variable a predecir
X = df_players.drop(columns = target).copy() # conjunto X para predecir
y = np.where(pd.isnull(df_players[target]), 0, 1) # conjunto y a ser predicho

In [6]:
print(f"La base cuenta con {X.shape[0]} filas")

La base cuenta con 17588 filas


Para procesar los datos, es necesario distinguir entre los tipos de variables del dataset. Se observa que, en su mayoría, son variables numéricas de tipo `int` y unas pocas variables `category`. 

In [7]:
X.dtypes.value_counts() # distribución de clases target

int64       31
category     2
category     1
category     1
category     1
category     1
category     1
dtype: int64

En el caso de las variables `category`, es necesario hacer la distinción entre las variables categóricas y ordinales, pues esto nos dirá el método apropiado de vectorizar estas variables. 

In [8]:
X.select_dtypes('category') # vista a las variables categóricas

,Name,Nationality,Club_Position,Preffered_Foot,Work_Rate,Weak_foot,Skill_Moves
0,Cristiano Ronaldo,Portugal,LW,Right,High / Low,4,5
1,Lionel Messi,Argentina,RW,Left,Medium / Medium,4,4
2,Neymar,Brazil,LW,Right,High / Medium,5,5
3,Luis Suárez,Uruguay,ST,Right,High / Medium,4,4
4,Manuel Neuer,Germany,GK,Right,Medium / Medium,4,1
...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,Sub,Right,Medium / Medium,1,1
17584,Dylan McGoey,Republic of Ireland,Sub,Right,Medium / Medium,2,1
17585,Tommy Ouldridge,England,Res,Right,High / Medium,2,2
17586,Mark Foden,Scotland,Sub,Right,Medium / Medium,3,1


In [9]:
X['Name'].value_counts() # notamos que la variable Name tiene valores repetidos, por lo tanto deja de ser una variable "id" y puede contener información relevante

Felipe                 6
Gabriel                5
Danilo                 5
Roberto                4
Carlos Rodríguez       4
                      ..
Guillermo Cubillos     1
Guillermo Farré        1
Guillermo Fernández    1
Guillermo Ferracuti    1
Łukasz Wiech           1
Name: Name, Length: 17341, dtype: int64

Notamos que las variables *Weak_foot* y *Skill_Moves* son variables ordinales. Por esto, es necesario hacer la distinción:

In [10]:
ordinal_vars = ['Weak_foot', 'Skill_Moves'] # variables ordinales
categoric_vars = [col for col in X.select_dtypes('category').columns if col not in ordinal_vars] # variables categóricas, descontamos las variables ordinales
numeric_vars = [col for col in X.select_dtypes('int').columns] # variables numéricas

Un aspecto importante a notar es que la base cuenta con 1 solo valor nulo en la columna *Club_position*. Por esto, se incluirá un paso de tratamiento de valores nulos en los `Pipeline` propuestos.

In [11]:
X.isnull().sum().sort_values(ascending = False)

Club_Position        1
Name                 0
Jumping              0
Long_Pass            0
Acceleration         0
Speed                0
Stamina              0
Strength             0
Balance              0
Agility              0
Heading              0
Crossing             0
Shot_Power           0
Finishing            0
Long_Shots           0
Curve                0
Freekick_Accuracy    0
Penalties            0
Short_Pass           0
Composure            0
Nationality          0
Vision               0
Height               0
Weight               0
Preffered_Foot       0
Age                  0
Work_Rate            0
Weak_foot            0
Skill_Moves          0
Ball_Control         0
Dribbling            0
Marking              0
Sliding_Tackle       0
Standing_Tackle      0
Aggression           0
Reactions            0
Interceptions        0
Volleys              0
dtype: int64

#### Creación de ColumnTransformer

Teniendo todo esto en consideración, los pasos a seguir para procesar los datos serán:

- Si la variable es numérica, imputar datos faltantes mediante `KNNImputer` (notar que el dataset no contiene valores nulos numéricos, se programa este paso por completitud). Luego, transformar los datos mediante `RobustScaler`.

- Si la variable es categórica, imputar datos faltantes mediante `SimpleImputer` y rellenando con el string `Null`. Luego, transformar los datos mediante `OneHotEncoder`.

- Si la variable es ordinal, imputar datos faltantes mediante `SimpleImputer` y rellenando con la categoría `NA`. Luego, transformar los datos mediante `OrdinalEncoder`.

Para implementar los anterior, se usará un `ColumnTransformer` compuesto de tres `Pipeline`: uno para las variables numéricas, otro para las variables categóricas y otro para las variables ordinales.

In [12]:
ordinal_cat = [['NA'] + [category for category in X[var].cat.categories] for var in ordinal_vars]
ordinal_cat

[['NA', '1', '2', '3', '4', '5'], ['NA', '1', '2', '3', '4', '5']]

In [13]:
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer, KNNImputer

numeric_pipe = Pipeline([
                         ('imputer', KNNImputer()),
                         ('scaler', RobustScaler()),
                         ])

categoric_pipe = Pipeline([
                         ('imputer', SimpleImputer(strategy = 'constant', fill_value = 'Null')),
                         ('onehot', OneHotEncoder(handle_unknown='ignore')),
                         ])

ordinal_pipe = Pipeline([
                         ('imputer', SimpleImputer(strategy = 'constant', fill_value = 'NA')),
                         ('encoder', OrdinalEncoder(categories = ordinal_cat)),
                         ])

preprocessor = ColumnTransformer([
                                  ('numeric', numeric_pipe, numeric_vars),
                                  ('categoric', categoric_pipe, categoric_vars),
                                  ('ordinal', ordinal_pipe, ordinal_vars)
])

#### Creación de Pipelines

Usando la guía sugerida, respondemos paso a paso:

- $>50$ samples? Sí (contamos con 17588 filas)

- predicting a category? Sí (si el jugador es seleccionado nacional o no)

- do you have labeled data? Sí (las etiquetas se generaron al principio del lab)

- $<100 \text{K}$ samples? Sí (contamos conn 17588 filas)

Respondiendo todas esas preguntas y en base a la guía, se usarán los clasificadores `LinearSVC`, `KNeighborsClassifier` y `SVC`. 

In [14]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

linearsvc_pipe = Pipeline([
                  ('preprocessing', preprocessor),
                  ('clf', LinearSVC()),
                  ])

knn_pipe = Pipeline([
            ('preprocessing', preprocessor),
            ('clf', KNeighborsClassifier()),
            ])

svc_pipe = Pipeline([
            ('preprocessing', preprocessor),
            ('clf', SVC()),
            ])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

#### Separación train test

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .33, shuffle = True, stratify = y)

Chequeamos proporciones `y_train`, `y_test`

In [16]:
pd.DataFrame(y_train).value_counts() / y_train.shape[0]

0    0.938895
1    0.061105
dtype: float64

In [17]:
pd.DataFrame(y_test).value_counts() / y_test.shape[0]

0    0.938846
1    0.061154
dtype: float64

Concluimos que `train_test_split` hace su trabajo y ambos conjuntos tienen la misma proporción de clases :)

#### Entrenando pipelines

In [18]:
linearsvc_pipe.fit(X_train, y_train)
y_pred = linearsvc_pipe.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      5450
           1       0.64      0.33      0.44       355

    accuracy                           0.95      5805
   macro avg       0.80      0.66      0.71      5805
weighted avg       0.94      0.95      0.94      5805



In [19]:
knn_pipe.fit(X_train, y_train)
y_pred = knn_pipe.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5450
           1       0.54      0.08      0.15       355

    accuracy                           0.94      5805
   macro avg       0.74      0.54      0.56      5805
weighted avg       0.92      0.94      0.92      5805



In [20]:
svc_pipe.fit(X_train, y_train)
y_pred = svc_pipe.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5450
           1       1.00      0.00      0.01       355

    accuracy                           0.94      5805
   macro avg       0.97      0.50      0.49      5805
weighted avg       0.94      0.94      0.91      5805



**Respuesta**:

*¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?*

En general, se aprecia un buen accuracy (en torno a .95), una buena precision y un mal recall para la clase positiva. Considerando que:

$$Precision = \frac{\text{Verdaderos Positivos}}{\text{Verdaderos positivos} + \text{Falsos positivos}}$$

$$Recall = \frac{\text{Verdaderos Positivos}}{\text{Verdaderos positivos} + \text{Falsos negativos}}$$

necesariamente implica que los falsos positivos de los modelos son muy pocos (cercanos a 0), mientras que los falsos negativos son muchos. En otras palabras, el modelo esta prediciendo muchos ejemplos que efectivamente son positivos (seleccionados nacionales) como negativos (que no lo son). Lo anterior es esperable, pues tal como se explica mas adelante, esto es fruto de tratar con un dataset desbalanceado. Por lo tanto, se concluye que los modelos son muy buenos prediciendo que un jugador no es seleccionado nacional pero muy malos prediciendo a los jugadores que sí son seleccionados nacionales.

*¿Podrían mejorarse los resultados? ¿Cómo?*

Existen muchas formas de mejorar estos resultados, entre estas se encuentran:

- Rebalanceo de clases: El principal problema de nuestros modelos es el desbalanceo de clases, por esto es posible recurrir a diferentes técnicas que puedan balancear las clases y obtener predicciones menos sesgadas. Entre estas técnicas se encuentran: `Undersampling`, `Oversampling`, `SMOTE`.
- Selección de variables: Seleccionar las *k* variables más relevantes puede ayudar a evitar la maldición de la dimensionalidad y por ende, permitir que el modelo aprenda mejor. Para esto, es posible usar filtrar mediante scores como `Chi2` y `F-score`.
- Reducción de dimensionalidad: En la misma linea que la Selección de variables, reducir la dmiensionalidad del problema permite que evitar la maldición de la dimensionalidad y así mejorar el aprendizaje de los modelos. Esto se puede lograr mediante la implementación de técnicas como `PCA` y `TSNE`
- Optimización de hiperparámetros: Es posible mejorar el rendimiento de los modelos mediante la optimización de los hiperparámetros de cada modelo. Para esto, es necesario usar técnicas de prueba-error como `GridSearch`.

*¿Influye la cantidad de ejemplos por clase?*

Tal como se describió anteriormente, las clases están desbalanceadas. Esto es perjudicial pues que una clase este sobrerepresentada induce a que el modelo esté inclinado a predecir esa clase y no la clase minoritaria. En ese sentido, si se quiere obtener un buen poder de predicción en ambas clases, es necesario fijarse en métricas como que puedan balancear ambos resultados, como el `F1-score`.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

#### Re etiquetamiento de target y count

In [21]:
map_dict = {
    'ataque': ['ST', 'CF'],
    'central_ataque': ['RW', 'CAM', 'LW'],
    'central': ['RM', 'CM', 'LM'],
    'central_defensa': ['RWB', 'CDM', 'LWB'],
    'defensa': ['RB', 'CB', 'LB'],
    'arquero': ['GK'],
    np.nan: ['Sub', 'Res'],
}

In [22]:
df_positions = df_players.copy() # copy de la data original
df_positions['Club_Position'] = df_positions['Club_Position'].apply(lambda x: [key for key, value in map_dict.items() if x in value][0] # agrupamos en categorias
                                    if len([key for key, value in map_dict.items() if x in value]) > 0 else np.nan) # seteamos a valores nulos valores no definidos
df_positions = df_positions.dropna(subset = ['Club_Position']) # dropeamos clases Sub, Res y todas aquellas que no esten contenidas en las definidas por el enunciado
df_positions['Club_Position'].value_counts() # count de clases

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Club_Position, dtype: int64

In [48]:
print(f"Se cuentan con {df_positions.shape[0]} observaciones!")

Se cuentan con 3939 observaciones!


Notamos que las clases están desbalanceadas: vemos que la clase "defensa" tiene una frecuencia 5 veces mayor que "central_defensa".

Además, notamos que por el drop de valores nulos contamos con muchas menos observaciones que para el caso anterior (3.939 observaciones).

#### Entrenamiento de Pipeline

In [23]:
target = 'Club_Position' # variable a predecir
X = df_positions.drop(columns = target).copy() # conjunto X
y = df_positions[target] # conjunto y

Notamos que ahora el problema es un problema de clasificación multiclase. Pese a esto, la guía provista nos sugiere lo mismo: usar `LinearSVC`, `KNeighborsClassifier` y `SVC`.

In [24]:
ordinal_vars = ['Weak_foot', 'Skill_Moves'] # variables ordinales
categoric_vars = [col for col in X.select_dtypes('category').columns if col not in ordinal_vars] # variables categóricas, descontamos las variables ordinales
numeric_vars = [col for col in X.select_dtypes('int').columns] # variables numéricas

numeric_pipe = Pipeline([
                         ('imputer', KNNImputer()),
                         ('scaler', RobustScaler()),
                         ])

categoric_pipe = Pipeline([
                         ('imputer', SimpleImputer(strategy = 'constant', fill_value = 'Null')),
                         ('onehot', OneHotEncoder(handle_unknown='ignore')),
                         ])

ordinal_pipe = Pipeline([
                         ('imputer', SimpleImputer(strategy = 'constant', fill_value = 'NA')),
                         ('encoder', OrdinalEncoder(categories = ordinal_cat)),
                         ])

preprocessor = ColumnTransformer([
                                  ('numeric', numeric_pipe, numeric_vars),
                                  ('categoric', categoric_pipe, categoric_vars),
                                  ('ordinal', ordinal_pipe, ordinal_vars)
])

In [25]:
linearsvc_pipe = Pipeline([
                  ('preprocessing', preprocessor),
                  ('clf', LinearSVC()),
                  ])

knn_pipe = Pipeline([
            ('preprocessing', preprocessor),
            ('clf', KNeighborsClassifier()),
            ])

svc_pipe = Pipeline([
            ('preprocessing', preprocessor),
            ('clf', SVC()),
            ])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .33, shuffle = True, stratify = y)

In [27]:
linearsvc_pipe.fit(X_train, y_train)
y_pred = linearsvc_pipe.predict(X_test)
print(classification_report(y_test, y_pred))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.80      0.88      0.84       142
        central       0.58      0.59      0.59       299
 central_ataque       0.54      0.39      0.45       192
central_defensa       0.47      0.25      0.32        69
        defensa       0.80      0.94      0.86       389

       accuracy                           0.74      1300
      macro avg       0.70      0.67      0.68      1300
   weighted avg       0.73      0.74      0.73      1300



In [28]:
knn_pipe.fit(X_train, y_train)
y_pred = knn_pipe.predict(X_test)
print(classification_report(y_test, y_pred))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.75      0.82      0.78       142
        central       0.52      0.61      0.56       299
 central_ataque       0.46      0.28      0.35       192
central_defensa       0.41      0.19      0.26        69
        defensa       0.80      0.90      0.85       389

       accuracy                           0.71      1300
      macro avg       0.66      0.63      0.63      1300
   weighted avg       0.69      0.71      0.69      1300



In [29]:
svc_pipe.fit(X_train, y_train)
y_pred = svc_pipe.predict(X_test)
print(classification_report(y_test, y_pred))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       209
         ataque       0.83      0.88      0.85       142
        central       0.58      0.69      0.63       299
 central_ataque       0.56      0.29      0.38       192
central_defensa       0.69      0.16      0.26        69
        defensa       0.80      0.96      0.87       389

       accuracy                           0.75      1300
      macro avg       0.74      0.66      0.66      1300
   weighted avg       0.74      0.75      0.73      1300



**Respuesta**:

Tal como en el caso anterior, es posible mejorar los resultados mediante diferentes técnicas, entre las que se encuentran: 

- Rebalanceo de clases: El principal problema de nuestros modelos es el desbalanceo de clases, por esto es posible recurrir a diferentes técnicas que puedan balancear las clases y obtener predicciones menos sesgadas. Entre estas técnicas se encuentran: `Undersampling`, `Oversampling`, `SMOTE`.
- Selección de variables: Seleccionar las *k* variables más relevantes puede ayudar a evitar la maldición de la dimensionalidad y por ende, permitir que el modelo aprenda mejor. Para esto, es posible usar filtrar mediante scores como `Chi2` y `F-score`.
- Reducción de dimensionalidad: En la misma linea que la Selección de variables, reducir la dmiensionalidad del problema permite que evitar la maldición de la dimensionalidad y así mejorar el aprendizaje de los modelos. Esto se puede lograr mediante la implementación de técnicas como `PCA` y `TSNE`
- Optimización de hiperparámetros: Es posible mejorar el rendimiento de los modelos mediante la optimización de los hiperparámetros de cada modelo. Para esto, es necesario usar técnicas de prueba-error como `GridSearch`.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

#### Explicación de $R^2$

El $R^2$ es un coeficiente que toma valores entre 0-1 y es interpretrado como varianza explicada por el modelo. El $R^2$ puede ser calculado por diferentes formas equivalentes:

$$R^2 = \frac{\sum_{i=1}^n (\hat y_i - \bar y)^2}{\sum_{i=1}^n (y_i - \bar y)^2} = 1 - \frac{\text{var}(\epsilon)}{\text{var}(y)} = \text{cor}(y, \hat y)^2$$

De esta forma, el $R^2$ toma valores cercanos a 1 cuanto más alta sea la calidad del modelo y valores cercanos a 0 cuanto más lejanas sean sus predicciones.

Un aspecto importante a notar es que el $R^2$ puede tomar valores negativos, lo que indica que el modelo ofrece predicciones peores a usar la media $\bar y$ para todas las precciones.

#### Generar Pipeline

In [30]:
df_sueldos = pd.read_csv(path + 'salarios.csv', index_col = 0)
df_sueldos

,Player,Weekly Salary
1528,Lionel Messi,1538905.0
1006,Cristiano Ronaldo,1248536.0
1529,Antoine Griezmann,997032.0
2004,Neymar,797726.0
1530,Gareth Bale,607706.0
...,...,...
1527,Simone Ghidotti,283.0
2539,Valentin Gendrey,100.0
1001,John Yeboah,70.0
1000,Antonis Aidonis,70.0


In [31]:
df_wages = df_players.copy()
df_wages = df_wages.merge(df_sueldos, left_on = 'Name', right_on = 'Player', how = 'inner') # hacemos merge con dataframe de sueldos
df_wages

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Player,Weekly Salary
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,85,92,93,90,81,76,85,88,Cristiano Ronaldo,1248536.0
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,71,85,95,88,89,90,74,85,Lionel Messi,1538905.0
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,62,78,89,77,79,84,81,83,Neymar,797726.0
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,77,87,94,86,86,84,85,88,Luis Suárez,508923.0
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,25,25,13,16,14,11,47,11,Manuel Neuer,326233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,Phillip Menzel,Germany,NaN,Res,191,83,Right,18,Medium / Medium,3,...,10,25,5,10,12,8,15,5,Phillip Menzel,2034.0
1862,Manuel Akanji,Switzerland,NaN,Sub,187,85,Right,21,Medium / Medium,2,...,51,38,25,18,26,27,40,31,Manuel Akanji,54176.0
1863,Moritz Nicolas,Germany,NaN,Res,195,87,Right,19,Medium / Medium,2,...,16,22,9,10,17,14,17,11,Moritz Nicolas,2262.0
1864,Giacomo Satalino,Italy,NaN,Sub,188,70,Right,17,Medium / Medium,1,...,11,19,6,5,13,11,18,9,Giacomo Satalino,2827.0


In [49]:
print(f"El dataset cuenta con {df_wages.shape[0]} observaciones!")

El dataset cuenta con 1866 observaciones!


Notamos que ahora, fruto del merge con el dataframe de sueldos, contamos con aún menos observaciones (1.866 observaciones).

In [32]:
target = 'Weekly Salary' # variable a predecir
X = df_wages.drop(columns = target).copy() # conjunto X
y = df_wages[target] # conjunto y

In [33]:
ordinal_vars = ['Weak_foot', 'Skill_Moves'] # variables ordinales
categoric_vars = [col for col in X.select_dtypes('category').columns if col not in ordinal_vars] # variables categóricas, descontamos las variables ordinales
numeric_vars = [col for col in X.select_dtypes('int').columns] # variables numéricas

# pipeline variables numericas
numeric_pipe = Pipeline([
                         ('imputer', KNNImputer()),
                         ('scaler', RobustScaler()),
                         ])

# pipeline variables categoricas
categoric_pipe = Pipeline([
                         ('imputer', SimpleImputer(strategy = 'constant', fill_value = 'Null')),
                         ('onehot', OneHotEncoder(handle_unknown='ignore')),
                         ])

# pipeline variables ordinales
ordinal_pipe = Pipeline([
                         ('imputer', SimpleImputer(strategy = 'constant', fill_value = 'NA')),
                         ('encoder', OrdinalEncoder(categories = ordinal_cat)),
                         ])

# juntamos todo en ColumnTransformer
preprocessor = ColumnTransformer([
                                  ('numeric', numeric_pipe, numeric_vars),
                                  ('categoric', categoric_pipe, categoric_vars),
                                  ('ordinal', ordinal_pipe, ordinal_vars)
])

In [44]:
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from xgboost import XGBRegressor

# pipeline ridge
ridge_pipe = Pipeline([
                  ('preprocessing', preprocessor),
                  ('reg', Ridge()),
                  ])

# pipeline linear svr
linearsvr_pipe = Pipeline([
            ('preprocessing', preprocessor),
            ('reg', SVR(kernel = 'linear')),
            ])

# pipeline svr
svr_pipe = Pipeline([
            ('preprocessing', preprocessor),
            ('reg', SVR(kernel = 'rbf')),
            ])

# pipeline xgboost
xgb_pipe = Pipeline([
            ('preprocessing', preprocessor),
            ('reg', XGBRegressor()),
            ])

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .33, shuffle = True) # separación en conjunto train y test

#### Obtener $R^2 > 0.35$

In [37]:
from sklearn.metrics import r2_score

ridge_pipe.fit(X_train, y_train)
y_pred = ridge_pipe.predict(X_test)
r2_score(y_test, y_pred)

0.18031676298687682

Desempeño positivo pero podría ser mejor, aún estamos distantes del 0.35 propuesto

In [38]:
linearsvr_pipe.fit(X_train, y_train)
y_pred = linearsvr_pipe.predict(X_test)
r2_score(y_test, y_pred)

-0.050596693327488396

In [39]:
svr_pipe.fit(X_train, y_train)
y_pred = svr_pipe.predict(X_test)
r2_score(y_test, y_pred)

-0.07369353873508122

Notamos que los modelos `SVR` tienen rendimiento muy malo puesto que sus prediccinoes arrojan un $R^2$ negativo

In [46]:
xgb_pipe.fit(X_train, y_train)
y_pred = xgb_pipe.predict(X_test)
r2_score(y_test, y_pred)

[20:56:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.5606466306856799

Notamos que el modelo `XGBoost` supera con bastaaaaante facilidad el 0.35 propuesto.

**Respuesta**:

En vista de los resultados, se concluye que en general los modelos ofrecen un mal ajuste a los datos. La excepción a esto es el modelo `XGBoost` que, tal como se espera al pertenecer a los modelos de ensamblaje, obtiene un rendimiento muy superior a los otros modelos. En ese sentido, mientras que los modelos `SVR` conseguían un $R^2$ negativo (generaban predicciones peores a la media de la variable objetivo), `XGBoost` obtiene predicciones que superan con bastante facilidad el 0.35 propuesto. 

Para mejorar el modelo, una regla general es **mientras más datos, mejor**. En ese sentido, tener más datos siempre será igual o mejor a tener menos datos, por lo que es una buena vía para obtener un mejor rendimiento. 

Usar diferentes características (por medio de *Feature Engineering*) también es una buena opción, puesto que al usar modelos que no tengan incorporado *Representation Learning* (es decir, la capacidad de aprender features en su entrenamiento), es necesario construir buenas features "a mano" para que de esta forma el modelo reciba estas features y las pueda usar para obtener un mejor ajuste.

Finalmente, ambas opciones no son excluyentes y tanto combinándolas como por sí solas indican buenas formas de mejorar el poder de predicción del modelo.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>